In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer

In [ ]:
df = pd.read_csv('train.csv')
df.rename(columns = {'signup_date':'days_active'}, inplace=True)
df['days_active'] = -1*df['days_active']
df.head()

,customer_id,age,location,subscription_type,payment_plan,num_subscription_pauses,payment_method,customer_service_inquiries,days_active,weekly_hours,average_session_length,song_skip_rate,weekly_songs_played,weekly_unique_songs,num_favorite_artists,num_platform_friends,num_playlists_created,num_shared_playlists,notifications_clicked,churned
0,1,32,Montana,Free,Yearly,2,Paypal,Medium,1606,22.391362,105.394516,0.176873,169,109,18,32,52,35,46,0
1,2,64,New Jersey,Free,Monthly,3,Paypal,Low,2897,29.294210,52.501115,0.981811,55,163,44,33,12,25,37,1
2,3,51,Washington,Premium,Yearly,2,Credit Card,High,348,15.400312,24.703696,0.048411,244,117,20,129,50,28,38,0
3,4,63,California,Family,Yearly,4,Apple Pay,Medium,2894,22.842084,83.595480,0.035691,442,252,47,120,55,17,24,0
4,5,54,Washington,Family,Monthly,3,Paypal,High,92,23.151163,52.578266,0.039738,243,230,41,66,40,32,47,0


#### Let's do age deature engineering

In [ ]:
df_copy = df.copy()

In [ ]:
df_copy.drop(columns = ['location','payment_method','customer_id','payment_plan'], inplace=True)
inquiry_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
df_copy['customer_service_inquiries'] = df_copy['customer_service_inquiries'].map(inquiry_mapping)

sub_mapping = {'Free': 0, 'Student': 1, 'Family': 2, 'Premium': 3}
df_copy['subscription_type'] = df_copy['subscription_type'].map(sub_mapping)

Payment plan showd no signal on analysis. Then it was dropped.

In [ ]:
### Cols to Engineer

# 'age', 'num_platform_friends', 'num_shared_playlists', 'notifications_clicked',

# 'weekly_songs_played', 'weekly_unique_songs','average_session_length','num_favorite_artists',
# 'num_playlists_created'

### Age Engineering

In [ ]:
bins = [18, 25, 35, 45, 55, 65, 80]
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65-79']

df_copy['age_group'] = pd.cut(df_copy['age'], bins=bins, labels=labels, right=False)

risk_mapping = {
    '18-24': 2,
    '65-79': 2,
    '25-34': 1,
    '55-64': 1,
    '35-44': 0,
    '45-54': 0
}
df_copy['age_group_risk'] = df_copy['age_group'].astype(str).map(risk_mapping)
# df_copy.drop(columns = ['age','age_group'], inplace=True)

df_copy[['age_group_risk','churned']].corr()
## This correlation is better than it looks, since in social settings, correlation are multivaritate and rarely above 0.50.


,age_group_risk,churned
age_group_risk,1.00000,0.17797
churned,0.17797,1.00000


Analysis of notification engagement:

In [ ]:
def segment_notifications(x):
    if x < 5:
        return 'Non-Engager'
    elif x < 45:
        return 'Standard'
    else:
        return 'Extreme'

df_copy['notif_segment'] = df_copy['notifications_clicked'].apply(segment_notifications)
notif_map = {'Non-Engager': 0, 'Standard': 1, 'Extreme': 2}
df_copy['notif_segment'] = df_copy['notif_segment'].map(notif_map)
df_copy[['notif_segment','churned']].corr()

## This feature will also be considrerd.

,notif_segment,churned
notif_segment,1.000000,-0.059762
churned,-0.059762,1.000000


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_copy.head()

,age,subscription_type,num_subscription_pauses,customer_service_inquiries,days_active,weekly_hours,average_session_length,song_skip_rate,weekly_songs_played,weekly_unique_songs,num_favorite_artists,num_platform_friends,num_playlists_created,num_shared_playlists,notifications_clicked,churned,age_group,age_group_risk,notif_segment
0,32,0,2,1,1606,22.391362,105.394516,0.176873,169,109,18,32,52,35,46,0,25-34,1,2
1,64,0,3,0,2897,29.294210,52.501115,0.981811,55,163,44,33,12,25,37,1,55-64,1,1
2,51,3,2,2,348,15.400312,24.703696,0.048411,244,117,20,129,50,28,38,0,45-54,0,1
3,63,2,4,1,2894,22.842084,83.595480,0.035691,442,252,47,120,55,17,24,0,55-64,1,1
4,54,2,3,2,92,23.151163,52.578266,0.039738,243,230,41,66,40,32,47,0,45-54,0,2


In [ ]:
# Check raw correlations
cols_to_check = ['weekly_songs_played', 'weekly_unique_songs']

I did grouping ans saw churn rate there.

 Engineered features like weekly explorer (by week_unique_song/week_song_played) It was fruitless.

Time spent  on unique songs, by week_hours, week_unique_song, week_song_played. It was multicollinear more with week_hours and less with churned.

Did direct churning correlation.

I finally tried cutting off outliers too and then perform all the correlation analysis. It did not make anything better.

Finally decided to drop the weekly columns.

In [ ]:
df_copy.drop(columns = cols_to_check, inplace=True)

Social, extrovertdess index. Having a group of friends, how much active is a person among them.

In [ ]:
# Assuming you have already created the _tier columns (0, 1, 2) for these
df_copy['extrovertness_index'] = (
    df_copy['num_shared_playlists'].div(df_copy['num_platform_friends']+1))
## If a person is using it as primary social media, then he'she won't churn. But a celebrity/influencer type person
# who doesn't, might. So it makes sense.
# Check correlation
print(df_copy['extrovertness_index'].corr(df_copy['churned']))

## we're keepint this.

0.01005235871315194


No signal at all was found in payment_plan.

Feature optimized so far:

Age, subscription_type,

num_shared_playlists, num_subscription_pauses,

customer_service_inquiries, weekly_hours,weekly_unique_songs

num_platform_friends, notification_clicked,
    
song_skip_rate, num_platform_friends,

num_shared_playlists.

Features to tackle:

days_active, average_session_length,

weekly_songs_played, num_favorite_artists, num_playlist_created.


In [ ]:
df_copy.drop(columns = ['days_active','average_session_length','num_favorite_artists',
'num_playlists_created'], inplace=True)

### Dropping original columns here.

In [ ]:
df_copy.drop(columns = ['age_group','age','notifications_clicked','num_shared_playlists','num_platform_friends'], inplace=True)

In [ ]:
df_copy.head()

,subscription_type,num_subscription_pauses,customer_service_inquiries,weekly_hours,song_skip_rate,churned,age_group_risk,notif_segment,extrovertness_index
0,0,2,1,22.391362,0.176873,0,1,2,1.060606
1,0,3,0,29.294210,0.981811,1,1,1,0.735294
2,3,2,2,15.400312,0.048411,0,0,1,0.215385
3,2,4,1,22.842084,0.035691,0,1,1,0.140496
4,2,3,2,23.151163,0.039738,0,0,2,0.477612


We did correlation, plotting, with hue analysis of all the "num" columns individually with churn. No signal was found.

In [ ]:
### Let's wrap up feature engineering here. Rest we'll decide on model training.